# RSNA-2025 ベースライン評価 (exp0001_baseline)

このノートブックでは、学習済みモデルの評価とOut-of-Fold予測を実行します。

- 実験ID: exp0001_baseline
- モデル: 学習済みGradientBoostingClassifier
- 評価指標: AUC、閾値最適化、混同行列

## 主な内容
1. 学習済みモデルのロード
2. Out-of-Fold（OOF）予測
3. 閾値最適化
4. 評価指標の可視化


In [ ]:
# 0) セットアップ
import os
import sys
import subprocess
from pathlib import Path
import pickle
import json

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib', 'gcsfs', 'fsspec'], check=True)
    from google.colab import auth
    auth.authenticate_user()
    os.chdir('/content')
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')
    sys.path.insert(0, str(Path.cwd() / 'src'))

GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)


In [ ]:
# 1) モデルとメタデータのロード
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

models_dir = Path('models/exp0001_baseline')

# モデルのロード
with open(models_dir / 'gbm_baseline.pkl', 'rb') as f:
    gbm = pickle.load(f)

# メタデータのロード
with open(models_dir / 'metadata.json', 'r') as f:
    metadata = json.load(f)

print("Model loaded successfully")
print(f"Training AUC: {metadata['train_auc']:.4f}")
print(f"Validation AUC: {metadata['val_auc']:.4f}")
print(f"Feature columns: {metadata['feature_columns'][:5]}...")


In [ ]:
# 2) データの再読み込みと特徴量作成
train_uri = f'{GCS_BASE}/train.csv'
train = pd.read_csv(train_uri, storage_options={'token': 'cloud'})

# 特徴量の再作成（training.ipynbと同じ処理）
df_age_str = train['PatientAge'].astype(str)
age_first = df_age_str.str.split(' - ').str[0]
age_vals = pd.to_numeric(age_first.str.extract(r'([0-9]+(?:\.[0-9]+)?)')[0], errors='coerce')

x_age = age_vals.fillna(age_vals.median())
X = pd.DataFrame({
    'age': x_age,
    'sex': (train['PatientSex'] == 'Male').astype(int)
})
mod_dummies = pd.get_dummies(train['Modality'], prefix='mod')
X = pd.concat([X, mod_dummies], axis=1)

# 目的変数
if train['Aneurysm Present'].dtype != np.int64 and train['Aneurysm Present'].dtype != np.int32:
    y = train['Aneurysm Present'].astype(int)
else:
    y = train['Aneurysm Present']

print(f"Data loaded: {X.shape}")


In [ ]:
# 3) Out-of-Fold (OOF) 予測
from sklearn.model_selection import StratifiedKFold

SEED = metadata['seed']
n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
oof_preds = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold = X.iloc[train_idx]
    y_train_fold = y.iloc[train_idx]
    X_val_fold = X.iloc[val_idx]
    
    # フォールドごとにモデルを学習
    from sklearn.ensemble import GradientBoostingClassifier
    fold_model = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=8,
        random_state=SEED
    )
    fold_model.fit(X_train_fold, y_train_fold)
    
    # OOF予測
    oof_preds[val_idx] = fold_model.predict_proba(X_val_fold)[:, 1]
    
    # フォールドごとのAUC
    fold_auc = roc_auc_score(y.iloc[val_idx], oof_preds[val_idx])
    print(f"Fold {fold+1} AUC: {fold_auc:.4f}")

# 全体のOOF AUC
overall_oof_auc = roc_auc_score(y, oof_preds)
print(f"\nOverall OOF AUC: {overall_oof_auc:.4f}")


In [ ]:
# 4) ROC曲線と閾値最適化
import matplotlib.pyplot as plt
import seaborn as sns

fpr, tpr, thresholds = roc_curve(y, oof_preds)

# Youden's J統計量で最適閾値を探索
youden_j = tpr - fpr
optimal_idx = np.argmax(youden_j)
optimal_threshold = thresholds[optimal_idx]

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {overall_oof_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=100, 
           label=f'Optimal threshold = {optimal_threshold:.3f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Aneurysm Present')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Optimal Threshold: {optimal_threshold:.3f}")
print(f"Sensitivity at optimal: {tpr[optimal_idx]:.3f}")
print(f"Specificity at optimal: {1-fpr[optimal_idx]:.3f}")


In [ ]:
# 5) 混同行列と分類レポート
from sklearn.metrics import ConfusionMatrixDisplay

y_pred_binary = (oof_preds >= optimal_threshold).astype(int)

# 混同行列の表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 正規化なし
cm = confusion_matrix(y, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Aneurysm', 'Aneurysm'])
disp.plot(ax=axes[0], cmap='Blues')
axes[0].set_title('Confusion Matrix (Count)')

# 正規化あり
cm_norm = confusion_matrix(y, y_pred_binary, normalize='true')
disp_norm = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=['No Aneurysm', 'Aneurysm'])
disp_norm.plot(ax=axes[1], cmap='Blues')
axes[1].set_title('Confusion Matrix (Normalized)')

plt.tight_layout()
plt.show()

# 分類レポート
print("\nClassification Report:")
print(classification_report(y, y_pred_binary, target_names=['No Aneurysm', 'Aneurysm']))


In [ ]:
# 6) OOF予測の保存
outputs_dir = Path('outputs/oof/exp0001_baseline')
outputs_dir.mkdir(parents=True, exist_ok=True)

# OOF予測をDataFrameとして保存
oof_df = pd.DataFrame({
    'SeriesInstanceUID': train['SeriesInstanceUID'],
    'Aneurysm Present': y,
    'oof_pred': oof_preds
})

oof_df.to_csv(outputs_dir / 'oof_predictions.csv', index=False)

# 評価メトリクスの保存
metrics = {
    'overall_oof_auc': float(overall_oof_auc),
    'optimal_threshold': float(optimal_threshold),
    'sensitivity': float(tpr[optimal_idx]),
    'specificity': float(1-fpr[optimal_idx])
}

with open(outputs_dir / 'metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"OOF predictions saved to: {outputs_dir}")
print(f"Overall OOF AUC: {overall_oof_auc:.4f}")
